# Lấy nội dung bài báo từ website và phân loại từng từ trong bài báo

Các bước : Phân tích dạng html của trang web --> Lấy file html của website --> Lấy nội dung của bài báo --> Phân loại từ loại

Chọn một bài báo bất kỳ trong trang web Dân trí : https://dantri.com.vn


*   Dễ thấy được nội dung 1 bài báo nào cũng được bắt đầu từ thẻ **<h1 class =" dt-news__title"**

*   Kết thúc ở thẻ **<div id="tin-lien-quan"**




#1. Import các thư viện cần thiết



In [ ]:
# Lấy source html của trang web
import urllib3

#Tìm kiếm trong python
import re  

# Chuẩn hóa các ký tự đặc biệt có trong text ( vd: &, vv...)
import html

# Phân loại danh từ, động từ,... trong text
from pyvi import ViTokenizer, ViPosTagger

print("import ok")

import ok


# 2. Lấy file html của trang web

Using: https://urllib3.readthedocs.io/en/latest/

In [ ]:
http = urllib3.PoolManager()
# Sử dụng hàm request trong urllib3 + phương thức GET ( lấy ) dữ liệu html
r = http.request('GET', 
                 'https://dantri.com.vn/xa-hoi/lu-rut-anh-ve-nhung-khong-con-chao-ai-duoc-nua-20201023115203360.htm')

# Chuẩn hóa dữ liệu sang UTF-8
txt = r.data.decode('utf-8')
print("HTML ok")

HTML ok


# 3. Tách lấy nội dung bài báo từ txt

Sử dụng biến:


*   start: đánh dấu vị trí bắt đầu của bài báo
*   first: đánh dấu vị trí bắt đầu từng đoạn text cần lấy
*   second: đánh dấu vị trí kết thúc từng đoạn text
*   data: array lưu nội dung bài báo cần lấy

Trong bài báo có chèn ảnh nên ta phải loại bỏ đi ảnh + nội dung (text) của ảnh:


*   text: kiểm tra nếu đang trong thẻ của ảnh **<figcaption** thì không lấy data





In [ ]:
def GetData(txt):
  start = txt.find('<h1 class="dt-news__title">', 0)
  # print(start)

  test = False
  data = []

  while True:
    #tim vt đầu tiên xuất hiện '>' sau vt start
    first = txt.find('>', start)
    # print("first", first)
    if first == -1: # nếu không tìm thấy thì thoát
      break

     #tìm vt đầu tiên xuất hiện '<' sau vt first
    second = txt.find('<', first)
    # print(second)
    if second == -1: # nếu không tìm thấy thì thoát
      break
      
    #nội dung cần lấy có dạng: ">...abc...<", trong đó vt '>' và vt '<' đã tìm thấy: first, second
    sub = txt[first+1:second].strip()
    
    #kiểm tra xem đoạn đầu có phải thuộc các thẻ không trong nội dung không ( bao gồm: phần ngoài nội dung bài báo + phần ảnh trong bài báo) 
    check = txt[start:first+1].strip()
    
    if check.find('id="tin-lien-quan"') != -1:
        break
    if check.find('<figcaption') !=-1:
        test = True
    if check.find('</figcaption') !=-1:
        test = False
    if len(sub) > 0 and test == False:
        data.append(sub)
    start = second #update lại start là vị trí
  return data

li = GetData(txt)
for i in li:
  print(i)


Lũ rút, anh về nhưng… không còn chào ai được nữa!
Dân trí
Sáng 23/10, đường dẫn vào nhà ông Ngô Bá Cường (SN 1960, xã Thạch Thắng, huyện Thạch Hà, Hà Tĩnh) nước đã rút, nắng đã lên, nhưng không khí vô cùng buồn thương khi đón liệt sĩ Ngô Bá Văn trở về.
Đón người con quê hương
Tối ngày 22/10, tại Đài hóa thân hoàn vũ xã Lưu Vĩnh Sơn, huyện Thạch Hà (Hà Tĩnh), lãnh đạo Quân khu 4, tỉnh Hà Tĩnh và các sở ngành, địa phương... đã tới dâng hương tiễn đưa các liệt sĩ Đoàn Kinh tế - Quốc phòng 337 hy sinh trong khi làm nhiệm vụ giúp Nhân dân khắc phục hậu quả mưa lũ tại tỉnh Quảng Trị về nơi an nghỉ cuối cùng.
Trước đó rạng sáng 18/10, tại thôn Cợp, xã Hướng Phùng, huyện Hướng Hóa, tỉnh Quảng Trị xảy ra vụ sạt lở đất nghiêm trọng. Đây là nơi ở của 27 cán bộ, chiến sĩ của Đoàn Kinh tế - Quốc phòng 337 đang thực hiện nhiệm vụ giúp dân khắc phục hậu quả mưa lũ. Vụ sạt lở đất đã khiến 22 cán bộ, chiến sĩ hy sinh, trong đó có Thượng úy quân nhân chuyên nghiệp Ngô Bá Văn (sinh ngày 8/1/1984, tại thô

#4. Phân loại các từ trong câu



Tách dữ liệu thành từng câu: Sau dấu chấm "." hoặc dấu hỏi "?" được tính là kết thúc câu.

Phân loại từng từ loại trong mỗi câu sử dụng thư viện Pyvi: https://pypi.org/project/pyvi/ :


* A - Adjective
* C - Coordinating conjunction
* E - Preposition
* I - Interjection
* L - Determiner
* M - Numeral
* N - Common noun
* Nc - Noun Classifier
* Ny - Noun abbreviation
* Np - Proper noun
* Nu - Unit noun
* P - Pronoun
* R - Adverb
* S - Subordinating conjunction
* T - Auxiliary, modal words
* V - Verb
* X - Unknown
* F - Filtered out (punctuation)





In [ ]:
for p in li:
    # chuẩn hóa ký tự đặc biệt nếu trong câu có ký tự đặc biệt
    text= html.unescape( p )

    # Tách dữ liệu thành từng câu: 
    arr = re.compile(" (?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s ").split( text )

    #Phân loại từng từ 
    for sentence in arr:
        # x = ViTokenizer.tokenize( sentence )
        xx =  ViPosTagger.postagging(ViTokenizer.tokenize( sentence ))
        for i in xx:
          print(i)

['Lũ', 'rút', ',', 'anh', 'về', 'nhưng', '…', 'không', 'còn', 'chào', 'ai', 'được', 'nữa', '!']
['N', 'V', 'F', 'N', 'V', 'C', 'F', 'R', 'V', 'V', 'P', 'V', 'R', 'F']
['Dân_trí']
['N']
['Sáng', '23', '/', '10', ',', 'đường_dẫn', 'vào', 'nhà', 'ông', 'Ngô_Bá_Cường', '(', 'SN', '1960', ',', 'xã', 'Thạch_Thắng', ',', 'huyện', 'Thạch_Hà', ',', 'Hà_Tĩnh', ')', 'nước', 'đã', 'rút', ',', 'nắng', 'đã', 'lên', ',', 'nhưng', 'không_khí', 'vô_cùng', 'buồn', 'thương', 'khi', 'đón', 'liệt_sĩ', 'Ngô', 'Bá_Văn', 'trở', 'về', '.']
['N', 'M', 'F', 'M', 'F', 'V', 'V', 'N', 'Nc', 'Np', 'F', 'Ny', 'M', 'F', 'N', 'Np', 'F', 'N', 'Np', 'F', 'Np', 'F', 'N', 'R', 'V', 'F', 'N', 'R', 'V', 'F', 'C', 'N', 'R', 'A', 'V', 'N', 'V', 'N', 'Np', 'Np', 'V', 'V', 'F']
['Đón', 'người', 'con', 'quê_hương']
['V', 'N', 'N', 'N']
['Tối_ngày', '22', '/', '10', ',', 'tại', 'Đài_hóa_thân', 'hoàn_vũ', 'xã', 'Lưu_Vĩnh_Sơn', ',', 'huyện', 'Thạch_Hà', '(', 'Hà_Tĩnh', ')', ',', 'lãnh_đạo', 'Quân_khu', '4', ',', 'tỉnh', 'Hà_Tĩnh', '